Perspectivas sobre Clima Extremo e Desastres Naturais

## Visão Geral
Esta é uma ferramenta abrangente projetada para fornecer análises aprofundadas e insights sobre condições climáticas extremas e riscos de desastres naturais. Utilizando técnicas avançadas de análise de dados e visualização, este projeto visa capacitar os usuários com o conhecimento para tomar decisões informadas diante das incertezas climáticas.

Este projeto foi desenvolvido durante a Imersão Alura I.A, demonstrando a aplicação prática de inteligência artificial na análise de dados climáticos.

### Trecho de código é responsável pela instalação dos pacotes necessários para executar o projeto.

In [1]:
!pip install requests fpdf2
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 2.8 MB/s eta 0:00:00


### Trecho de código é responsável pela importação de bibliotecas

In [2]:
from datetime import datetime

import google.generativeai as genai
import matplotlib.pyplot as plt
import requests
import os
from fpdf import FPDF
from google.colab import userdata


### Responsável pela configuração da inteligência artificial (IA) fornecida pela Google. Ao instalar a biblioteca google-generativeai

In [3]:
api_key = userdata.get('API_KEY')
genai.configure(api_key=api_key)

generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}


safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]


# Geração de Relatório de Riscos Climáticos

Este bloco de código é responsável por criar um relatório de riscos climáticos em formato PDF. Ele define uma classe `PDF` personalizada para configurar o cabeçalho e rodapé do documento, limpa o conteúdo de marcações Markdown, gera um gráfico de temperatura versus tempo com base nos dados fornecidos e, finalmente, compila tudo em um arquivo PDF estruturado e formatado.

- `class PDF(FPDF)`: Define o layout do PDF, incluindo cabeçalho e rodapé.
- `clean_markdown(content)`: Remove símbolos de Markdown do texto.
- `create_temperature_graph(hourly_data, temperature_data)`: Cria um gráfico visualizando a variação da temperatura ao longo do tempo.
- `create_pdf(info, content)`: Compila as informações e o gráfico em um PDF e salva o arquivo.

In [21]:
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 16)
        self.cell(0, 10, 'Relatório de Riscos', 0, 1, 'C')
        if self.page_no() == 1:
            self.image('./static/logo.png', 10, 8, 33)
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')


def clean_markdown(content):
    content = content.replace('*', '').replace('#', '')
    return content


def create_temperature_graph(hourly_data, temperature_data):
    time_dt = [datetime.fromisoformat(t) for t in hourly_data['time']]

    plt.figure(figsize=(10, 6))
    plt.plot(time_dt, temperature_data, marker='o', linestyle='-', color='b')
    plt.xlabel('Hora')
    plt.ylabel('Temperatura (°C)')
    plt.title('Variação da Temperatura ao Longo do Tempo')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()

    graph_filename = 'temperatura_vs_tempo.png'
    plt.savefig(graph_filename)
    plt.close()
    return graph_filename


def create_pdf(info, content):
    pdf = PDF()
    pdf.add_page()

    pdf.set_font("Arial", 'B', 14)
    pdf.set_fill_color(230, 240, 255)

    pdf.cell(0, 10, txt=f"Nome da Localidade: {info['city']}", ln=True, fill=True)
    pdf.cell(0, 10, txt=f"Data de Criação: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)

    pdf.set_font("Arial", size=12)

    cleaned_content = clean_markdown(content)
    pdf.multi_cell(0, 10, txt=cleaned_content)

    graph_filename = create_temperature_graph(info['hourly'], info['hourly']['temperature_2m'])
    pdf.image(graph_filename, x=10, y=None, w=190)

    if not os.path.exists('generated_pdf'):
        os.makedirs('generated_pdf')

    pdf_file_name = f'generated_pdf/Relatorio_riscos_{info["latitude"]}_{info["longitude"]}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pdf'

    pdf.output(pdf_file_name)


# Obtenção de Localização por IP

A função `get_location_by_ip` é utilizada para obter a localização geográfica do usuário com base no endereço IP atual. Ela faz uma solicitação à API `ip-api.com` e, em caso de sucesso, retorna um dicionário contendo a latitude, longitude e cidade. Se ocorrer um erro na solicitação, uma mensagem de erro é exibida e a função retorna `None`.


In [22]:
def get_location_by_ip():
    try:
        response = requests.get('http://ip-api.com/json')
        response.raise_for_status()
        data = response.json()
        return {
            'latitude': data.get('lat'),
            'longitude': data.get('lon'),
            'city': data.get('city')
        }
    except requests.RequestException as e:
        print(f'Erro ao obter localização por IP: {e}')
        return None


# Recuperação de Dados Meteorológicos

A função `get_weather_data` é projetada para recuperar dados meteorológicos com base em coordenadas geográficas específicas. Ela realiza uma solicitação GET para a API Open-Meteo, solicitando previsões horárias de temperatura. Se a solicitação for bem-sucedida, a função retorna os dados em formato JSON. Caso contrário, ela captura a exceção, imprime uma mensagem de erro e retorna `None`.


In [23]:
def get_weather_data(latitude, longitude):
    try:
        weather_api_url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m'
        response = requests.get(weather_api_url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f'Erro ao obter dados da API de previsão do tempo: {e}')
        return None


# Criação de instrução

A função `instruction_ia` é projetada para gerar uma instrução que guiará a criação de um relatório profissional, utilizando as informações fornecidas.


In [24]:
def instruction_ia(info):
  instruction = (
      "Utilize as seguintes informações para criar um relatório profissional, destacando os seguintes tópicos:\n\n"
      f"    Latitude: {info['latitude']}\n"
      f"    Longitude: {info['longitude']}\n"
      f"    Tempo de Geração (ms): {info['generationtime_ms']}\n"
      f"    Deslocamento UTC (segundos): {info['utc_offset_seconds']}\n"
      f"    Fuso Horário: {info['timezone']}\n"
      f"    Abreviação do Fuso Horário: {info['timezone_abbreviation']}\n"
      f"    Elevação: {info['elevation']}\n\n"
      "O relatório deve abordar os seguintes aspectos:\n\n"
      "    Riscos: Identificação de possíveis perigos ou ameaças na região relacionados a DESASTRES NATURAIS.\n"
      "    Condições Climáticas Extremas: Avaliação de temperaturas extremas, ventos fortes, ou outras condições meteorológicas adversas.\n"
      "    Medidas de Prevenção: Sugestões de ações preventivas que os usuários podem adotar para se protegerem desses Riscos e Condições.\n"
  )
  return instruction


# Geração de Relatório Meteorológico Automatizado

Este trecho de código é responsável por automatizar a geração de um relatório meteorológico profissional.

In [25]:
location_info = get_location_by_ip()
if location_info:
    info = get_weather_data(
        location_info['latitude'],
        location_info['longitude']
    )

    info['city'] = location_info['city']
    model = genai.GenerativeModel(
      model_name="gemini-1.5-pro-latest",
      generation_config=generation_config,
      safety_settings=safety_settings
    )

    content = model.generate_content(instruction_ia(info))

    content_text = content._result.candidates[0].content.parts[0].text
    create_pdf(info, content_text)

<ipython-input-21-b1a86abcd2a6>:3: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 16)
<ipython-input-21-b1a86abcd2a6>:4: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Relatório de Riscos', 0, 1, 'C')
<ipython-input-21-b1a86abcd2a6>:42: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", 'B', 14)
<ipython-input-21-b1a86abcd2a6>:45: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 10, txt=f"Nome da Localidade: {info['city']}", ln=True, fill=True)
<ipython-input-21-b1a86abcd2a6>:45: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, txt=f"Nome da Localida